## Google authentication

In [11]:
import os
import tensorflow as tf
import json
from google.colab import auth
auth.authenticate_user()

use_tpu = 'COLAB_TPU_ADDR' in os.environ
if use_tpu:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)
  
  with tf.Session(TPU_ADDRESS) as session:
    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.
else:
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

TPU address is grpc://10.31.118.34:8470


## Download *improv* project

In [12]:
import sys

!test -d improv || git clone https://github.com/rikhuijzer/improv improv
if not 'improv' in sys.path:
  sys.path += ['improv']
%cd improv
!git log -n 1 --format='commit %H'
%cd ..

/content/improv
commit eaaabe8543d3c2c1f6fdb0a229e4bad55a625e34
/content


In [13]:
!test -d nlu_datasets || git clone https://github.com/rikhuijzer/nlu_datasets nlu_datasets
if not 'nlu_datasets' in sys.path:
  sys.path += ['nlu_datasets']
%cd nlu_datasets
!git log -n 1 --format='commit %H'
%cd ..

/content/nlu_datasets
commit 12e6fe28bf4ad4ba9ebc17f6a1c053ff9de62b5e
/content


In [14]:
!pip install rasa_nlu

## BERT pre-trained 


In [0]:
bucket = 'benchmark-tpu-bucket' #@param {type:"string"}
assert bucket, 'Must specify an existing GCS bucket name'
bucket_models_dir = 'gs://{}/bert/models'.format(bucket)
bert_model = 'BERT-Large, Uncased' #@param ['BERT-Base, Uncased', 'BERT-Large, Uncased', 'BERT-Large, Cased', 'BERT-Large, Multilingual Cased']

# lowest two are unavailable in cloud-tpu-checkpoints
# need to rewrite to pull them from google storage apis
bert_model_map = {
    'BERT-Base, Uncased': 'uncased_L-12_H-768_A-12',
    'BERT-Large, Uncased': 'uncased_L-24_H-1024_A-16',
    'BERT-Large, Cased': 'cased_L-24_H-1024_A-16',
    'BERT-Large, Multilingual Cased': 'multi_cased_L-12_H-768_A-12'
}
bert_model_mapped = bert_model_map[bert_model]

bert_pretrained_dir = 'gs://cloud-tpu-checkpoints/bert/' + bert_model_mapped

## Define params

In [0]:
from improv.config import HParams
from improv.utils import get_project_root
from pathlib import Path
from datetime import datetime, timedelta

corpus = 'chatbot'  #@param ['chatbot', 'askubuntu', 'webapplications', 'snips2017']
task = 'intent' #@param ['ner', 'intent', 'ner_intent']
num_train_steps = 250  #@param {type:"integer"}
train_batch_size = 32  #@param {type:"integer"}
new_dir = True  #@param {type:"boolean"}
if new_dir:
  output_dir_name = str(datetime.now() + timedelta(hours=1))[:-7]
tpu_name = 'grpc://' + os.environ['COLAB_TPU_ADDR'] if use_tpu else ''
do_train = True  #@param {type:"boolean"}
do_eval = True  #@param {type:"boolean"}
do_predict = True  #@param {type:"boolean"}

h_params = HParams(
  data_dir=Path('nlu_datasets') / 'generated' / corpus / task,
  bert_config_file=os.path.join(bert_pretrained_dir, 'bert_config.json'),
  task=task,
  task_name=corpus,
  vocab_file=os.path.join(bert_pretrained_dir, 'vocab.txt'),
  output_dir=bucket_models_dir + '/{}/{}'.format(corpus, output_dir_name),
  local_dir=str(get_project_root() / 'tmp' / corpus / output_dir_name),
  init_checkpoint=os.path.join(bert_pretrained_dir, 'bert_model.ckpt'),
  do_lower_case=bert_model.startswith('uncased'),
  max_seq_length=128,
  do_train_eval=False,
  do_train=do_train,
  do_eval=do_eval,
  do_predict=do_predict,
  train_batch_size=train_batch_size,
  eval_batch_size=8,
  predict_batch_size=8,
  learning_rate=5e-5,
  num_train_steps=num_train_steps,
  warmup_proportion=0.1,
  save_checkpoints_steps=50,  # ignored by TPUEstimator
  save_summary_steps=50,  # ignored by TPUEstimator
  iterations_per_loop=1000,  # ignored by TPUEstimator
  use_tpu=use_tpu,
  tpu_name=tpu_name,
  tpu_zone=None,
  gcp_project=None,
  master=None,
  num_tpu_cores=8
)

tf.gfile.MakeDirs(h_params.output_dir)
tf.gfile.MakeDirs(h_params.local_dir)

## Main


In [17]:
from improv.kyzhouhzau_ner import main, run

result = run(h_params)

INFO:tensorflow:Using config: {'_model_dir': 'gs://benchmark-tpu-bucket/bert/models/chatbot/2018-12-24 14:36:18', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': 50, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.31.118.34:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f83321745c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'grpc://10.31.118.34:8470', '_evaluation_master': b'grpc://10.31.118.34:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shard

In [18]:
h_params

Params(data_dir=PosixPath('nlu_datasets/generated/chatbot/intent'), bert_config_file='gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_config.json', task='intent', task_name='chatbot', vocab_file='gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/vocab.txt', output_dir='gs://benchmark-tpu-bucket/bert/models/chatbot/2018-12-24 14:36:18', local_dir='improv/tmp/chatbot/2018-12-24 14:36:18', init_checkpoint='gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt', do_lower_case=False, max_seq_length=128, do_train_eval=False, do_train=True, do_eval=True, do_predict=True, train_batch_size=32, eval_batch_size=8, predict_batch_size=8, learning_rate=5e-05, num_train_steps=250, warmup_proportion=0.1, save_checkpoints_steps=50, save_summary_steps=50, iterations_per_loop=1000, use_tpu=True, tpu_name='grpc://10.31.118.34:8470', tpu_zone=None, gcp_project=None, master=None, num_tpu_cores=8)

In [19]:
from improv.kyzhouhzau_ner import evaluate_ner_pred_result

evaluate_ner_pred_result(h_params, result)

FileNotFoundError: ignored

In [0]:
from pathlib import Path
from improv.evaluate import print_scores

print_scores(Path(h_params.local_dir) / 'results.txt')

In [20]:
from improv.utils import convert_result_pred
from sklearn.metrics import f1_score
from improv.my_classifier import get_unique_intents, get_data_filename, get_intents

def print_intent_f1(h_params: HParams, result):
  intents = get_unique_intents(get_data_filename(h_params))
  
  y_pred = []
  for pred in result:
    probs = pred['probabilities']
    id = [i for i, j in enumerate(probs) if j == max(probs)][0]
    y_pred.append(intents[id])
   
  y_true = get_intents(get_data_filename(h_params), training=False)
  score = f1_score(y_true, y_pred, average='weighted')
  score = round(score, 3)
  print('intents weighted f1: {}'.format(score))

print_intent_f1(h_params, result)

intents weighted f1: 0.991


In [0]:
from improv.my_classifier import evaluate, get_model_fn_and_estimator

model_fn, estimator = get_model_fn_and_estimator(h_params)
evaluate(h_params, estimator)